In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

#metrics
from statistics import mean
from sklearn.metrics import accuracy_score 
from sklearn.metrics import explained_variance_score as evs
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

Matplotlib is building the font cache; this may take a moment.


In [11]:
simd=pd.read_csv('../data/combined_dataset_cleaned.csv')

In [12]:
simd.head()

,Unnamed: 0,Data_Zone,Intermediate_Zone,Council_area,Total_population,Working_age_population_revised,Income_rate,Income_count,Employment_rate,Employment_count,...,PT_GP,PT_Post,PT_retail,crime_count,crime_rate,overcrowded_count,nocentralheat_count,overcrowded_rate,nocentralheat_rate,year
0,0,S01006506,Culter,Aberdeen City,904.0,605.0,0.07,60.0,0.07,40.0,...,8.437832,5.987087,5.711433,8.00642939150402,88.5666968086728,87.0,10.0,0.102113,0.011737,2016
1,1,S01006507,Culter,Aberdeen City,830.0,491.0,0.07,60.0,0.05,25.0,...,8.331833,7.262817,6.794300,4.00321469575201,48.2315023584579,85.0,4.0,0.101675,0.004785,2016
2,2,S01006508,Culter,Aberdeen City,694.0,519.0,0.05,30.0,0.03,15.0,...,7.853631,5.827924,5.251454,4.00321469575201,57.6832088725073,31.0,8.0,0.048212,0.012442,2016
3,3,S01006509,Culter,Aberdeen City,573.0,354.0,0.05,30.0,0.06,20.0,...,7.434491,8.311862,8.444698,*,*,42.0,6.0,0.072414,0.010345,2016
4,4,S01006510,Culter,Aberdeen City,676.0,414.0,0.10,70.0,0.07,30.0,...,5.141013,6.627376,6.619130,12.009644087256,177.657456912071,50.0,7.0,0.086655,0.012132,2016


In [13]:
simd.isnull().sum()

Unnamed: 0                        0
Data_Zone                         0
Intermediate_Zone                 0
Council_area                      0
Total_population                  0
Working_age_population_revised    0
Income_rate                       2
Income_count                      0
Employment_rate                   2
Employment_count                  0
CIF                               0
ALCOHOL                           0
DRUG                              0
SMR                               0
DEPRESS                           0
LBWT                              0
EMERG                             0
Attendance                        2
Attainment                        1
Noquals                           0
NEET                              3
HESA                              0
drive_petrol                      0
drive_GP                          0
drive_PO                          0
drive_primary                     0
drive_retail                      0
drive_secondary             

In [14]:
simd.columns

Index(['Unnamed: 0', 'Data_Zone', 'Intermediate_Zone', 'Council_area',
       'Total_population', 'Working_age_population_revised', 'Income_rate',
       'Income_count', 'Employment_rate', 'Employment_count', 'CIF', 'ALCOHOL',
       'DRUG', 'SMR', 'DEPRESS', 'LBWT', 'EMERG', 'Attendance', 'Attainment',
       'Noquals', 'NEET', 'HESA', 'drive_petrol', 'drive_GP', 'drive_PO',
       'drive_primary', 'drive_retail', 'drive_secondary', 'PT_GP', 'PT_Post',
       'PT_retail', 'crime_count', 'crime_rate', 'overcrowded_count',
       'nocentralheat_count', 'overcrowded_rate', 'nocentralheat_rate',
       'year'],
      dtype='object')

In [17]:
numeric_columns = simd.select_dtypes(include=['int', 'float']).columns.tolist()
numeric_columns

['Unnamed: 0',
 'Total_population',
 'Working_age_population_revised',
 'Income_rate',
 'Income_count',
 'Employment_rate',
 'Employment_count',
 'ALCOHOL',
 'DRUG',
 'SMR',
 'EMERG',
 'Noquals',
 'NEET',
 'drive_petrol',
 'drive_GP',
 'drive_PO',
 'drive_primary',
 'drive_retail',
 'drive_secondary',
 'PT_GP',
 'PT_Post',
 'PT_retail',
 'overcrowded_count',
 'nocentralheat_count',
 'overcrowded_rate',
 'nocentralheat_rate',
 'year']

In [21]:
simd[numeric_columns].describe()

,Unnamed: 0,Total_population,Working_age_population_revised,Income_rate,Income_count,Employment_rate,Employment_count,ALCOHOL,DRUG,SMR,...,drive_retail,drive_secondary,PT_GP,PT_Post,PT_retail,overcrowded_count,nocentralheat_count,overcrowded_rate,nocentralheat_rate,year
count,8042.000000,8042.000000,8042.000000,8040.000000,8042.000000,8040.000000,8042.000000,8042.000000,8042.000000,8042.000000,...,8042.000000,8042.000000,8042.000000,8042.000000,8042.000000,8042.000000,8042.000000,8042.000000,8042.000000,8042.000000
mean,4020.500000,773.620617,500.776424,0.121623,92.655558,0.105061,51.381870,99.330153,95.947311,98.990301,...,5.081690,5.945417,10.046098,8.469875,13.102799,83.870057,13.766600,0.111320,0.018261,2016.530216
std,2321.669765,200.509331,163.736946,0.095424,75.261552,0.079730,41.238569,99.790209,147.090533,45.677567,...,5.765077,4.896820,5.942590,4.386443,10.127055,65.128254,17.783708,0.079961,0.022527,1.356453
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.685475,1.017791,1.601271,1.930570,1.835432,0.000000,0.000000,0.000000,0.000000,2016.000000
25%,2010.250000,638.000000,400.000000,0.050000,35.000000,0.040000,20.000000,32.799203,0.000000,70.000000,...,2.777500,3.628994,6.267574,5.568977,7.823843,36.000000,3.000000,0.050000,0.004283,2016.000000
50%,4020.500000,758.000000,479.000000,0.100000,70.000000,0.080000,40.000000,70.165935,43.320199,92.000000,...,3.946242,4.776938,8.665514,7.465039,10.969180,68.000000,8.000000,0.093750,0.010345,2016.000000
75%,6030.750000,884.000000,569.000000,0.180000,130.000000,0.150000,70.000000,133.146010,129.245471,120.000000,...,5.730346,6.531299,12.007307,10.139799,15.499208,114.000000,17.000000,0.152297,0.023510,2016.000000
max,8041.000000,3847.000000,3423.000000,0.730000,555.000000,0.530000,325.000000,2350.544008,1864.126199,950.000000,...,190.000000,116.149193,108.789987,40.277934,190.000000,490.000000,187.000000,0.583882,0.214964,2020.000000


In [54]:
correlation_matrix = simd[numeric_columns].corr()
correlation=correlation_matrix[correlation_matrix > 0.5].fillna(0)
correlation


,Unnamed: 0,Total_population,Working_age_population_revised,Income_rate,Income_count,Employment_rate,Employment_count,ALCOHOL,DRUG,SMR,...,drive_retail,drive_secondary,PT_GP,PT_Post,PT_retail,overcrowded_count,nocentralheat_count,overcrowded_rate,nocentralheat_rate,year
Unnamed: 0,1.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.587325
Total_population,0.000000,1.00000,0.91505,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Working_age_population_revised,0.000000,0.91505,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Income_rate,0.000000,0.00000,0.00000,1.000000,0.937342,0.960425,0.887816,0.662814,0.602498,0.539333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.533016,0.000000,0.580562,0.000000,0.000000
Income_count,0.000000,0.00000,0.00000,0.937342,1.000000,0.894768,0.951849,0.638888,0.573193,0.523268,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.623356,0.000000,0.579494,0.000000,0.000000
Employment_rate,0.000000,0.00000,0.00000,0.960425,0.894768,1.000000,0.917073,0.684009,0.623805,0.526053,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.546828,0.000000,0.000000
Employment_count,0.000000,0.00000,0.00000,0.887816,0.951849,0.917073,1.000000,0.672955,0.599873,0.517058,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.645925,0.000000,0.602425,0.000000,0.000000
ALCOHOL,0.000000,0.00000,0.00000,0.662814,0.638888,0.684009,0.672955,1.000000,0.564774,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
DRUG,0.000000,0.00000,0.00000,0.602498,0.573193,0.623805,0.599873,0.564774,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
SMR,0.000000,0.00000,0.00000,0.539333,0.523268,0.526053,0.517058,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [68]:
# for i in range(0,correlation_matrix.shape[0]):
#     for j in range(0,correlation_matrix.shape[1]):
#         if(correlation_matrix.iloc[i,j]>0.5 and correlation_matrix.iloc[i,j]!=1):
#             print(numeric_columns[i],numeric_columns[j])
#             print(correlation_matrix.iloc[i,j])

In [71]:
final_dict={numeric_columns[i]+" "+numeric_columns[j]:correlation_matrix.iloc[i,j] for j in range(0,correlation_matrix.shape[1]) for i in range(0,correlation_matrix.shape[0]) if(correlation_matrix.iloc[i,j]>0.5 and correlation_matrix.iloc[i,j]!=1) }

In [80]:
for i,j in final_dict.items():
    print(i)
    print(j)
    print("***********")

year Unnamed: 0
0.5873248822074465
***********
Working_age_population_revised Total_population
0.9150502661038414
***********
Total_population Working_age_population_revised
0.9150502661038414
***********
Income_count Income_rate
0.9373415803320199
***********
Employment_rate Income_rate
0.9604246140056043
***********
Employment_count Income_rate
0.887815947474351
***********
ALCOHOL Income_rate
0.6628144873054533
***********
DRUG Income_rate
0.6024982079896563
***********
SMR Income_rate
0.5393327275902097
***********
EMERG Income_rate
0.8271858396158407
***********
Noquals Income_rate
0.8762713470112445
***********
NEET Income_rate
0.6825590038366052
***********
overcrowded_count Income_rate
0.5330158709627201
***********
overcrowded_rate Income_rate
0.5805616681960838
***********
Income_rate Income_count
0.9373415803320199
***********
Employment_rate Income_count
0.8947681018647816
***********
Employment_count Income_count
0.9518493692722477
***********
ALCOHOL Income_count
0.638888

In [86]:
simd['Employment_count'].unique()

array([ 40.,  25.,  15.,  20.,  30.,   5.,  10.,   0.,  65.,  55.,  35.,
        70.,  75.,  80.,  60.,  50.,  45.,  90., 110.,  95., 105., 100.,
        85., 120., 125., 115., 155., 145., 135., 165., 185., 210., 130.,
       225., 175., 140., 205., 150., 190., 160., 200., 170., 180., 220.,
       230., 195., 215., 235., 240., 305., 320., 325., 265., 270., 285.,
       250., 260., 197., 144., 183., 127., 124., 117., 172., 143., 111.,
       138., 137., 131., 178., 112., 121., 118., 116., 149.,  88., 211.,
       119., 101.,  84., 122.,  98.,  86.,  96.,  81.,  93.,  66., 128.,
        91., 106.,  83.,  77., 104., 113.,  78.,  94.,  97.,  73., 126.,
        99.,  76.,  82.,  72.,  87.,  74., 107., 102.,  61.,  68.,  79.,
        69.,  71.,  92.,  64.,  54.,  41.,  48., 103.,  63.,  58.,  53.,
        59.,  67.,  62.,  51.,  56.,  52.,  46.,  57.,  44.,  47.,  49.,
        38.,  31.,  42.,  33.,  43.,  39.,  34.,  28.,  36.,  26.,  32.,
        29.,  21.,  19.,  13.,  27.,  37.,  11.,  1